Mounted Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


import package

In [2]:
import tensorflow as tf

In [3]:
! pip install transformers torch speechrecognition
! pip install TTS torch
! pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 43.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 34.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 58.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 75.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at mult

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 25.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:
      Successfully uninstalled numpy-1.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires numpy==1.22.0; python_version <= "3.10", but you have numpy 1.23.5 which is incompatible.
albucore 0.0.19 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.20 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
bigframes 1.27.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.87 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
jax 0.4.33 requires numpy>

In [1]:
import numpy as np
import speech_recognition as sr
import torch
from TTS.api import TTS
import time
import os
import shutil

In [2]:
# Install the openai package
%pip install openai

from openai import OpenAI
api_key = 'sk-IE1KpyGZOJ6NBbwOkP1iPiIeyQVFj336gPP5DBC9XzIcGbfn'
client = OpenAI(api_key=api_key)

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="sk-IE1KpyGZOJ6NBbwOkP1iPiIeyQVFj336gPP5DBC9XzIcGbfn",
    base_url="https://api.chatanywhere.org"
)

Test script

In [3]:
from tensorflow.keras.models import Sequential, model_from_json

json_path = "/content/drive/MyDrive/ML Team 35/Codes/Chatbot/CNN_model.json" #google drive path
weights_path = "/content/drive/MyDrive/ML Team 35/Codes/Chatbot/CNN_model_weights.h5" #google drive path

with open(json_path, 'r') as json_file:
    loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights(weights_path)

print("Loaded model from disk")

Loaded model from disk


In [4]:
import pickle

scaler_path = '/content/drive/MyDrive/ML Team 35/Codes/Chatbot/scaler.pickle'
encoder_path = '/content/drive/MyDrive/ML Team 35/Codes/Chatbot/encoder.pickle'

with open(scaler_path, 'rb') as f:
    scaler2 = pickle.load(f)

with open(encoder_path, 'rb') as f:
    encoder2 = pickle.load(f)

print("Done")

Done


In [5]:
import librosa
def zcr(data,frame_length,hop_length):
    zcr=librosa.feature.zero_crossing_rate(data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(zcr)
def rmse(data,frame_length=2048,hop_length=512):
    rmse=librosa.feature.rms(y=data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(rmse)
def mfcc(data,sr,frame_length=2048,hop_length=512,flatten:bool=True):
    mfcc=librosa.feature.mfcc(y=data,sr=sr)
    return np.squeeze(mfcc.T)if not flatten else np.ravel(mfcc.T)

def extract_features(data,sr=22050,frame_length=2048,hop_length=512):
    result=np.array([])

    result=np.hstack((result,
                      zcr(data,frame_length,hop_length),
                      rmse(data,frame_length,hop_length),
                      mfcc(data,sr,frame_length,hop_length)
                     ))
    return result

In [6]:
def get_predict_feat(path):
    d, s_rate= librosa.load(path, duration=2.5, offset=0.6)
    res=extract_features(d)
    result=np.array(res)
    result=np.reshape(result,newshape=(1,2376))
    i_result = scaler2.transform(result)
    final_result=np.expand_dims(i_result, axis=2)

    return final_result

In [7]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
!pip -q install pydub
!apt-get install -y ffmpeg
from pydub import AudioSegment

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
audio = record(sec=5)

audio_path_colab = "/content/recorded_audio.wav"
audio.export(audio_path_colab, format="wav")
print(f"Audio saved to Colab local storage: {audio_path_colab}")

audio_path_drive = "/content/drive/MyDrive/ML Team 35/Codes/Chatbot/recorded_audio.wav"
audio.export(audio_path_drive, format="wav")
print(f"Audio also saved to Google Drive: {audio_path_drive}")


<IPython.core.display.Javascript object>

AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
from IPython.display import Audio
Audio(audio_path_drive)
Audio(audio_path_colab)

In [ ]:
emotions1={1:'Neutral', 2:'Calm', 3:'Happy', 4:'Sad', 5:'Angry', 6:'Fearful', 7:'Disgust',8:'Surprised'}
def prediction(path1):
    res=get_predict_feat(path1) #提取特征
    predictions=loaded_model.predict(res,verbose=0) #模型預測
    y_pred = encoder2.inverse_transform(predictions) #解碼預測結果
    return (y_pred[0][0])  #輸出情感類別

In [ ]:
print(prediction(audio_path_colab))

Language detection

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg


In [ ]:
import whisper

# 加载模型
model = whisper.load_model("base")


In [ ]:
# 加载音频文件
result = model.transcribe(audio_path_colab, task="language")  # task="language"用于检测语言
print("Detected Language:", result['language'])
speaker_language = result['language']
if speaker_language == "zh" :
  speaker_language = "zh-cn"

STT

In [ ]:
def speech_to_text(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio_data, language=speaker_language)
        return text
    except sr.UnknownValueError:
        return "Sorry, I couldn't understand the audio."
    except sr.RequestError:
        return "Sorry, there was a request error."

In [ ]:
stt_result = speech_to_text(audio_path_colab)
print(stt_result)

In [ ]:
def get_ser_label(audio_file):
    ser_result = prediction(audio_file)  # Replace with dynamic emotion inference in actual usage
    return ser_result

In [ ]:
speech_emotion = get_ser_label(audio_path_colab)
print(speech_emotion)

GPT 3.5

In [ ]:
def gpt_35_api(messages: list, language: str = "en", emotion: str = None):
    """为提供的对话消息创建新的回答，根据传入的语言参数和情绪决定回复。

    Args:
        messages (list): 完整的对话消息。
        language (str): 回复语言代码（如 "en", "zh", "es"）。
        emotion (str): 识别的用户情绪（如 "happy", "sad", "angry"）。
    """
    # 根据传入语言参数设置系统消息
    language_context = {
        "en": "You are an assistant that replies in English. Please respond with empathy considering the user's emotion.",
        "es": "Eres un asistente que responde en español. Responde con empatía considerando la emoción del usuario.",
        "fr": "Vous êtes un assistant qui répond en français. Répondez avec empathie en tenant compte de l'émotion de l'utilisateur.",
        "de": "Du bist ein Assistent, der auf Deutsch antwortet. Bitte antworte einfühlsam und berücksichtige die Emotion des Benutzers.",
        "it": "Sei un assistente che risponde en italiano. Rispondi con empatia considerando l'emozione dell'utente.",
        "pt": "Você é um assistente que responde em português. Responda com empatia considerando a emoção do usuário.",
        "pl": "Jesteś asystentem, który odpowiada po polsku. Odpowiedz z empatią, biorąc pod uwagę emocje użytkownika.",
        "tr": "Türkçe cevap veren bir asistansın. Kullanıcının duygusunu dikkate alarak empatiyle yanıt ver.",
        "ru": "Вы — помощник, отвечающий на русском языке. Пожалуйста, ответьте с сочувствием, учитывая эмоции пользователя.",
        "nl": "Je bent een assistent die in het Nederlands antwoordt. Reageer met empathie, rekening houdend met de emoties van de gebruiker.",
        "cs": "Jste asistent, který odpovídá v češtině. Odpovězte s empatií s ohledem na emoce uživatele.",
        "ar": "أنت مساعد يجيب باللغة العربية. يرجى الرد بتعاطف مع مراعاة مشاعر المستخدم.",
        "zh-cn": "你是一个使用中文回复的助手。请根据用户的情绪提供有同理心的回复。",
        "ja": "日本語で返信するアシスタントです。ユーザーの感情を考慮して共感的に返信してください。",
        "hu": "Ön egy asszisztens, aki magyarul válaszol. Kérem, válaszoljon empatikusan, figyelembe véve a felhasználó érzelmeit.",
        "ko": "한국어로 응답하는 어시스턴트입니다. 사용자의 감정을 고려하여 공감으로 응답하세요。",
        "hi": "आप एक सहायक हैं जो हिंदी में जवाब देता है। कृपया उपयोगकर्ता की भावना को ध्यान में रखते हुए सहानुभूति के साथ जवाब दें।"
    }

    system_message = language_context.get(language, "You are an assistant that replies in English. Please respond with empathy.")

    # 将情绪注入到上下文
    if emotion:
        system_message += f" The user's detected emotion is '{emotion}'. Please adjust your response accordingly."

    # 插入系统消息
    messages.insert(0, {"role": "system", "content": system_message})

    # 调用 GPT API
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo", messages=messages)
    reply = completion.choices[0].message.content
    return reply


def gpt_35_api_stream(messages: list, language: str = "en", emotion: str = None):
    """为提供的对话消息创建新的回答 (流式传输)，根据传入的语言参数和情绪决定回复。

    Args:
        messages (list): 完整的对话消息。
        language (str): 回复语言代码（如 "en", "zh", "es"）。
        emotion (str): 识别的用户情绪（如 "happy", "sad", "angry"）。
    """
    # 根据传入语言参数设置系统消息
    language_context = {
        "en": f"You are an assistant that replies in English. Please respond with empathy considering the user's emotion.",
        "zh-cn": f"你是一个使用中文回复的助手。请根据用户的情绪提供有同理心的回复。",
        "es": f"Eres un asistente que responde en español. Responde con empatía considerando la emoción del usuario.",
        "fr": f"Vous êtes un assistant qui répond en français. Répondez avec empathie en tenant compte de l'émotion de l'utilisateur.",
        "de": f"Du bist ein Assistent, der auf Deutsch antwortet. Bitte antworte einfühlsam und berücksichtige die Emotion des Benutzers.",
        "ja": f"日本語で返信するアシスタントです。ユーザーの感情を考慮して共感的に返信してください。",
        "ko": f"한국어로 응답하는 어시스턴트입니다. 사용자 감정을 고려하여 공감으로 응답하십시오。",
        "hi": f"आप एक सहायक हैं जो हिंदी में जवाब देता है। कृपया उपयोगकर्ता की भावना को ध्यान में रखते हुए सहानुभूति के साथ जवाब दें।"
    }
    system_message = language_context.get(language, "You are an assistant that replies in English. Please respond with empathy.")

    # 将情绪注入到上下文
    if emotion:
        system_message += f" The user's detected emotion is '{emotion}'. Please adjust your response accordingly."

    # 插入系统消息
    messages.insert(0, {"role": "system", "content": system_message})

    # 调用 GPT API (流式)
    reply = ""
    stream = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=messages,
        stream=True,
    )
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            content = chunk.choices[0].delta.content
            reply += content
    return reply


In [ ]:
def gpt_35_api_stream(messages: list, language: str = "en", emotion: str = None):
    """为提供的对话消息创建新的回答 (流式传输)，根据传入的语言参数和情绪决定回复。

    Args:
        messages (list): 完整的对话消息。
        language (str): 回复语言代码（如 "en", "zh", "es"）。
        emotion (str): 识别的用户情绪（如 "Happy", "Sad", "Angry"）。
    """
    # 根据传入语言参数设置系统消息
    language_context = {
        "en": f"You are an assistant that replies in English. Please respond with empathy considering the user's emotion.",
        "zh-cn": f"你是一个使用中文回复的助手。请根据用户的情绪提供有同理心的回复。",
        "es": f"Eres un asistente que responde en español. Responde con empatía considerando la emoción del usuario.",
        "fr": f"Vous êtes un assistant qui répond en français. Répondez avec empathie en tenant compte de l'émotion de l'utilisateur.",
        "de": f"Du bist ein Assistent, der auf Deutsch antwortet. Bitte antworte einfühlsam und berücksichtige die Emotion des Benutzers.",
        "ja": f"日本語で返信するアシスタントです。ユーザーの感情を考慮して共感的に返信してください。",
        "ko": f"한국어로 응답하는 어시스턴트입니다. 사용자 감정을 고려하여 공감으로 응답하십시오。",
        "hi": f"आप एक सहायक हैं जो हिंदी में जवाब देता है। कृपया उपयोगकर्ता की भावना को ध्यान में रखते हुए सहानुभूति के साथ जवाब दें।"
    }
    system_message = language_context.get(language, "You are an assistant that replies in English. Please respond with empathy.")

    # 将情绪注入到上下文并增强逻辑
    if emotion:
        emotion_guidelines = {
            "Neutral": "Respond normally based on the user's content.",
            "Calm": "Respond in a composed and supportive manner.",
            "Happy": "Share their joy and respond with an uplifting and enthusiastic tone.",
            "Sad": "Provide comforting and encouraging words to uplift their mood.",
            "Angry": "Help them calm down and encourage a rational perspective. Avoid escalating the situation.",
            "Fearful": "Offer reassurance and words of encouragement to help them overcome their fears.",
            "Disgust": "Address their concerns respectfully and provide supportive and understanding feedback.",
            "Surprised": "Encourage them to share more details and join in their excitement."
        }
        # 根据情绪类型调整系统消息
        emotion_instruction = emotion_guidelines.get(emotion, "Respond with empathy considering the user's emotion.")
        system_message += f" The user's detected emotion is '{emotion}'. {emotion_instruction}"

    # 插入系统消息
    messages.insert(0, {"role": "system", "content": system_message})

    # 调用 GPT API (流式)
    reply = ""
    stream = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=messages,
        stream=True,
    )
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            content = chunk.choices[0].delta.content
            reply += content
    return reply


In [ ]:
# 准备对话消息
summarizing_factor = "(reply in one sentence with more concerning with speaker's emotion)"
messages = [{'role': 'user', 'content': stt_result + summarizing_factor + speech_emotion}]

# 调用 GPT 流式 API
gpt_reply = gpt_35_api_stream(messages, language=speaker_language, emotion=speech_emotion)
print(gpt_reply, end="")

TTS

In [ ]:
# 检查设备是否支持 GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on device: {device}")

In [ ]:
model_name = "tts_models/multilingual/multi-dataset/xtts_v2"

tts = TTS(model_name).to(device)
print(f"Model {model_name} is loaded and ready.")


In [ ]:
TEXT = gpt_reply
EMOTION = speech_emotion  # "Happy", "Sad"
LANGUAGE = speaker_language  # "zh-cn"，"en"

output_dir_colab = "./output/"
os.makedirs(output_dir_colab, exist_ok=True)
output_file_path_colab = os.path.join(output_dir_colab, f"output_{EMOTION}.wav")

output_dir_drive = "/content/drive/MyDrive/ML Team 35/Codes/Chatbot/Generated_Audio/"
os.makedirs(output_dir_drive, exist_ok=True)
output_file_path_drive = os.path.join(output_dir_drive, f"output_{EMOTION}.wav")

start = time.perf_counter()
tts.tts_to_file(
    text=TEXT,
    speaker_wav="/content/drive/MyDrive/ML Team 35/Codes/Chatbot/LiShen.wav",
    language=LANGUAGE,
    file_path=output_file_path_colab,
    emotion=EMOTION,
)
end = time.perf_counter()

shutil.copy(output_file_path_colab, output_file_path_drive)

print(f"Audio file saved to Colab: {output_file_path_colab}")
print(f"Audio file also saved to Drive: {output_file_path_drive}")
print(f"Generation time: {end - start:.2f} seconds")


In [ ]:
from IPython.display import Audio

if os.path.exists(output_file_path_colab):
    print(f"Playing generated file: {output_file_path_colab}")
    display(Audio(output_file_path_colab))
else:
    print("Generated file not found. Please check the paths and settings.")
